In [ ]:
#pip show scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from time import time
from datetime import timedelta
import pickle as pkl

from joblib import dump
import os
import json

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from graphviz import Digraph

from sklearn.metrics import confusion_matrix, \
                  classification_report, accuracy_score,  precision_score, recall_score, f1_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
models_dir = '/content/drive/MyDrive/Machine Learning Deliverable/models'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning Deliverable/data/spotify_ml_21_5.csv')
df.shape

(37667, 14)

In [ ]:
df['genre'].value_counts()

genre
Underground Rap    5771
Dark Trap          4518
Hiphop             2954
trance             2734
techno             2632
psytrance          2610
dnb                2407
hardstyle          2277
trap               2226
techhouse          2192
RnB                2039
Trap Metal         1897
Rap                1809
Emo                1601
Name: count, dtype: int64

In [ ]:
def confusion(true, pred):
    """
    Function for pretty printing confusion matrices for every different class in our target
    """
    pred = pd.Series(pred)
    true = pd.Series(true)

    true.name = 'target'
    pred.name = 'predicted'
    cm = pd.crosstab(true.reset_index(drop=True), pred.reset_index(drop=True))
    cm = cm[cm.index]
    return cm

In [ ]:
np.random.seed(6046)

In [ ]:
# def split_data(df):
#   """
#   Function that reads a dataframe and splits data into training, validation and testing set. The split is stratified, so they distribution of the classes in our target column is equal.
#   """
#   X = df.loc[:, df.columns != 'genre']
#   y = df['genre']

#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=df.loc[:,'genre'])

#   X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, stratify=y_train, random_state=42)
#   return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
labels = list(df['genre'].unique())   # extract in a list all the different labels of our target genre column
#results_df = pd.DataFrame()   # this SHOULD READ the already existed dataframe, so it appends and not replace
results_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Deliverable/data/ML_results21_5.csv')
def compute_metrics(y_true, y_pred, labels, algorithm_name, results_df=None):
  """
  This function computes accuracy, F1 macro average on all classes, and F1 of every different class of our target.
  It also appends the results in a given dataframe. The parameters given are the predicted labels, the actual labels, the name of the algorithm to distinct the experiments and a datafarme.
  """
  accuracy = accuracy_score(y_true, y_pred)
  f1_scores = f1_score(y_true, y_pred, average=None, labels=labels)
  f1_score_avg = f1_score(y_true, y_pred, average='macro')

  # If results_df is not provided or empty, create a new DataFrame
  if results_df is None or results_df.empty:
      columns = ['Algorithm', 'Accuracy', 'F1_Avg'] + [f'F1_{label}' for label in labels]
      results_df = pd.DataFrame(columns=columns)

  # Append results to the DataFrame with algorithm name as index
  results_df.loc[len(results_df)] = [algorithm_name, accuracy, f1_score_avg] + list(f1_scores)

  return results_df

In [ ]:
results_df.head()

,Algorithm,Accuracy,F1_Avg,F1_Dark Trap,F1_Emo,F1_Trap Metal,F1_techhouse,F1_Underground Rap,F1_Hiphop,F1_psytrance,F1_techno,F1_hardstyle,F1_trap,F1_trance,F1_RnB,F1_dnb,F1_Rap
0,MLP-best,0.653939,0.659028,0.491882,0.653280,0.350299,0.841159,0.535264,0.459556,0.899317,0.826460,0.821138,0.771637,0.819218,0.374690,0.946984,0.435511
1,ExtrTrees-best2,0.638329,0.655252,0.469866,0.699862,0.329177,0.842528,0.450266,0.400000,0.910180,0.830287,0.851958,0.778797,0.830986,0.418511,0.961111,0.400000
2,ExtrTrees-best,0.637488,0.654126,0.469604,0.698061,0.326633,0.841683,0.450437,0.400000,0.911111,0.826923,0.851145,0.778455,0.831250,0.408493,0.960222,0.403743
3,MLP-default,0.641210,0.644428,0.502179,0.638239,0.341085,0.800817,0.529730,0.402053,0.883682,0.825234,0.800000,0.758985,0.781457,0.413260,0.929163,0.416107
4,ExtraTrees-default,0.619476,0.626704,0.454877,0.679831,0.246533,0.819939,0.478049,0.349398,0.883362,0.826873,0.809886,0.770031,0.781919,0.355342,0.938776,0.379045


In [ ]:
# X_train, X_test, X_val, y_train, y_test, y_val = split_data(df)
with open("/content/drive/MyDrive/Machine Learning Deliverable/data/split/train.pkl", "rb") as f:
    X_train, y_train = pkl.load(f)

with open("/content/drive/MyDrive/Machine Learning Deliverable/data/split/val.pkl", "rb") as f:
    X_val, y_val = pkl.load(f)

with open("/content/drive/MyDrive/Machine Learning Deliverable/data/split/test.pkl", "rb") as f:
    X_test, y_test = pkl.load(f)

In [ ]:
print(len(X_train)/len(df),len(X_val)/len(df), len(X_test)/len(df), len(df))

0.4488809833541296 0.22109538853638463 0.33002362810948577 37667


In [ ]:
scoring_dict = {
    'f1_mac': 'f1_macro',
    'acc': 'accuracy'
}

Gaussian Naive Bayes is a classification technique used in machine learning based on a probabilistic approach and Gaussian distribution. It assumes that each feature, has an independent capacity of predicting the output variable.

In [ ]:
gauss_nb = GaussianNB()
gauss_nb.fit(X_train,y_train)

dump(gauss_nb, os.path.join(models_dir, 'GaussianNB_default.joblib'))

y_pred = gauss_nb.predict(X_val)
results_df = compute_metrics(y_val, y_pred, labels, "GuassianNB-default", results_df)
print(confusion(y_val,y_pred))

predicted        Dark Trap  Emo  Hiphop  Rap  RnB  Trap Metal  \
target                                                          
Dark Trap              237  169      20   32   60          37   
Emo                      5  271       9    2   22           5   
Hiphop                  24   80     246   48  136           8   
Rap                      9   46      38  127   49           2   
RnB                     11  115      60   32  171           1   
Trap Metal              31   73      10   23   20          68   
Underground Rap         48  117     173  120  166          34   
dnb                      9   27      11    0    3           3   
hardstyle                8   37       0    0    1           2   
psytrance                2    0       0    0    0           0   
techhouse                3   14       5    2    4           3   
techno                  17    1       1    0    1           0   
trance                   3   32       4    2    6           0   
trap                     

In [ ]:
print(classification_report(y_val, y_pred))

                 precision    recall  f1-score   support

      Dark Trap       0.57      0.24      0.34       999
            Emo       0.27      0.77      0.39       354
         Hiphop       0.42      0.38      0.40       653
            Rap       0.33      0.32      0.32       400
            RnB       0.26      0.38      0.31       451
     Trap Metal       0.41      0.16      0.23       419
Underground Rap       0.47      0.46      0.46      1276
            dnb       0.96      0.90      0.93       532
      hardstyle       0.70      0.77      0.73       503
      psytrance       0.72      0.87      0.79       577
      techhouse       0.83      0.73      0.78       485
         techno       0.68      0.80      0.74       582
         trance       0.63      0.69      0.66       605
           trap       0.73      0.52      0.61       492

       accuracy                           0.55      8328
      macro avg       0.57      0.57      0.55      8328
   weighted avg       0.57   

The only hyperparameter we try to fine-tune with grid search (5-folds) is the var_smoothing, which is the portion of the largest variance of all features that is added to variances for calculation stability.

In [ ]:
scoring_cols = [
    'param_var_smoothing','mean_test_f1_mac','mean_test_acc'
]

In [ ]:
init_time = time()
gnb = GridSearchCV(estimator=gauss_nb,
                   scoring=scoring_dict,
                   param_grid={
                       'var_smoothing': np.logspace(0,-9, num=100)
                   },
                   cv=5,
                   return_train_score=False,
                   refit='f1_mac',
                   n_jobs=-1)
gnb_5CV = gnb.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

0:00:28.102715


In [ ]:
pd.DataFrame(gnb_5CV.cv_results_).sort_values(by='mean_test_f1_mac', ascending=False)[scoring_cols].head()
# we store all the results of the cross validation, sort them by mean F1 and print the 5 best along with the hyperparameters tested

,param_var_smoothing,mean_test_f1_mac,mean_test_acc
56,0.000008,0.549019,0.546960
59,0.000004,0.549011,0.546960
55,0.00001,0.549000,0.546960
60,0.000004,0.548995,0.546901
61,0.000003,0.548995,0.546901


In [ ]:
best_params = gnb_5CV.best_params_
best_params

{'var_smoothing': 8.111308307896871e-06}

In [ ]:
with open(os.path.join(models_dir, "GaussianNB_bestparams.txt"), "w") as f:
  f.write(json.dumps(best_params))
dump(gnb_5CV, os.path.join(models_dir, 'GaussianNB_best.joblib'))

['/content/drive/MyDrive/Machine Learning Deliverable/models/GaussianNB_best.joblib']

In [ ]:
df.head()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre,vocal_density,vocal_expression
0,0.774,0.503,5,-8.994,0.0442,0.2640,0.075700,0.0746,0.2710,140.003,146187,Dark Trap,0.880100,0.011978
1,0.583,0.669,0,-7.136,0.1440,0.0524,0.000007,0.0889,0.7500,159.966,157500,Emo,0.855993,0.108000
2,0.628,0.873,11,-3.476,0.3230,0.1820,0.001130,0.5580,0.1630,160.023,183405,Dark Trap,0.675870,0.052649
3,0.421,0.582,2,-8.257,0.0531,0.1980,0.377000,0.1490,0.0763,135.763,109394,Trap Metal,0.569900,0.004052
4,0.530,0.647,5,-17.322,0.0623,0.6840,0.871000,0.3720,0.1130,128.013,135000,Dark Trap,0.066700,0.007040


In [ ]:
y_pred = gnb_5CV.predict(X_val)

results_df = compute_metrics(y_val, y_pred, labels, "GuassianNB-best", results_df)

# confusion(y_val,y_pred)

In [ ]:
results_df.sort_values(by='F1_Avg', ascending=False)
results_df

,Algorithm,Accuracy,F1_Avg,F1_Dark Trap,F1_Emo,F1_Trap Metal,F1_techhouse,F1_Underground Rap,F1_Hiphop,F1_psytrance,F1_techno,F1_hardstyle,F1_trap,F1_trance,F1_RnB,F1_dnb,F1_Rap
0,GuassianNB-default,0.548151,0.549127,0.335694,0.394756,0.232479,0.779847,0.463560,0.399027,0.786834,0.736089,0.732580,0.608076,0.659341,0.311759,0.926214,0.321519
1,GuassianNB-best,0.548151,0.549050,0.335932,0.395044,0.232479,0.777656,0.463560,0.399027,0.786834,0.735504,0.733772,0.608076,0.658288,0.311759,0.927255,0.321519
2,LogisticRegression-default,0.556556,0.541332,0.427497,0.390310,0.268293,0.724696,0.523452,0.383731,0.791209,0.757601,0.580584,0.660277,0.624500,0.304348,0.757895,0.384252


Logistic regression is a supervised machine learning algorithm that accomplishes binary classification tasks by predicting the probability of an outcome, event, or observation. The model delivers a binary or dichotomous outcome limited to two possible outcomes: yes/no, 0/1, or true/false.
The multi_class hyperparameter is 'auto' and when it is auto and we have more than 2 classes it uses the 'multinomial' one. For ‘multinomial’ the loss minimised is the multinomial loss fit across the entire probability distribution and it uses the cross-entropy loss.

In [ ]:
lr = LogisticRegression(random_state=42, n_jobs=-1)
lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)
results_df = compute_metrics(y_val, y_pred, labels, "LogisticRegression-default", results_df)
print(confusion(y_val,y_pred))

predicted        Dark Trap  Emo  Hiphop  Rap  RnB  Trap Metal  \
target                                                          
Dark Trap              398   18      18    8   19          32   
Emo                     18  145       4    3   39           0   
Hiphop                  61   21     217   31   51           4   
Rap                     23    8      25  122   14           1   
RnB                     30   50      70   25  112           0   
Trap Metal              74   12       4    1    1          77   
Underground Rap        122   23     122   40   34          33   
dnb                     14   15      11    0    2           0   
hardstyle               38   51       0    0    3           4   
psytrance                6    1       0    0    1           0   
techhouse                6    7       3    3    3           0   
techno                   2    1       1    1    1           0   
trance                  40   13       0    0    4           1   
trap                    3

In [ ]:
dump(lr, os.path.join(models_dir, "LogisticRegression-default.joblib"))

['/content/drive/MyDrive/Machine Learning Deliverable/models/LogisticRegression-default.joblib']

Logistic Regression hyperparameters:


*   Penalty: This refers to the regularization. We test with L1 and L2 regularizations, i.e. lasso and ridge regularizations respectively. We also try 'None' so no regularization is tested. Lasso Regression (Least Absolute Shrinkage and Selection Operator) adds “absolute value of magnitude” of coefficient as penalty term to the loss function. Ridge regression adds “squared magnitude” of coefficient as penalty term to the loss function.
*   C: Inverse of regularization strength; must be a positive float. Smaller values specify stronger regularization. Values tested are [0.001,0.01,0.1,1,10].
*   Solver: Algorithm to use in the optimization problem. The three values tested are ['newton-cg', 'sag','lbfgs']. Newton’s method uses in a sense a better quadratic function minimisation. It's better because it uses the quadratic approximation. LBFGS (Limited-memory Broyden Fletcher Goldfarb Shanno) is an analogue of Newton’s Method, yet here the Hessian matrix is approximated using updates specified by gradient evaluations (or approximate gradient evaluations).The SAG method optimizes the sum of a finite number of smooth convex functions. Like stochastic gradient (SG) methods, the SAG method's iteration cost is independent of the number of terms in the sum. However, by incorporating a memory of previous gradient values, the SAG method achieves a faster convergence rate than black-box SG methods. All these 3 optimization algorithms work only with L2 or None regularization. That means that testing L1 is useless. So we take it out.
*   class_weight: Weights associated with classes. 'None' and 'balanced' are tested. The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))







In [ ]:
init_time = time()
lr_cv = GridSearchCV(estimator=lr,
                   scoring=scoring_dict,
                   param_grid={
                       'penalty': ['l2',None],
                       'C': [0.001,0.01,0.1,1,10],
                       'class_weight': [None,'balanced'],
                       'solver': ['newton-cg', 'sag','lbfgs']
                   },
                   cv=5,
                   return_train_score=False,
                   refit='f1_mac',
                   n_jobs=-1)
lr_5CV = lr_cv.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

0:07:07.315375


In [ ]:
scoring_cols = [
    'param_penalty', 'param_C', 'param_class_weight', 'param_solver' ,'mean_test_f1_mac','mean_test_acc'
]

In [ ]:
pd.DataFrame(lr_5CV.cv_results_).sort_values(by='mean_test_f1_mac', ascending=False)[scoring_cols].head()

,param_penalty,param_C,param_class_weight,param_solver,mean_test_f1_mac,mean_test_acc
4,None,0.001,None,sag,0.553862,0.565117
16,None,0.01,None,sag,0.553862,0.565117
40,None,1,None,sag,0.553862,0.565117
28,None,0.1,None,sag,0.553862,0.565117
52,None,10,None,sag,0.553862,0.565117


In [ ]:
best_params = lr_5CV.best_params_
best_params

{'C': 0.001, 'class_weight': None, 'penalty': None, 'solver': 'sag'}

In [ ]:
with open(os.path.join(models_dir, "LogisticRegression_bestparams.txt"), "w") as f:
  f.write(json.dumps(best_params))
dump(lr_5CV, os.path.join(models_dir, "LogisticRegression-best.joblib"))

['/content/drive/MyDrive/Machine Learning Deliverable/models/LogisticRegression-best.joblib']

In [ ]:
# results_df = results_df[results_df.Algorithm != 'LogisticRegression-best']

In [ ]:
y_pred = lr_5CV.predict(X_val)

results_df = compute_metrics(y_val, y_pred, labels, "LogisticRegression-best", results_df)
results_df.sort_values(by='F1_Avg', ascending=False,inplace=True)
results_df

,Algorithm,Accuracy,F1_Avg,F1_Dark Trap,F1_Emo,F1_Trap Metal,F1_techhouse,F1_Underground Rap,F1_Hiphop,F1_psytrance,F1_techno,F1_hardstyle,F1_trap,F1_trance,F1_RnB,F1_dnb,F1_Rap
3,LogisticRegression-best,0.565802,0.554464,0.433135,0.392573,0.341463,0.730382,0.520530,0.394366,0.829841,0.790102,0.587452,0.661856,0.657729,0.301691,0.759176,0.362205
0,GuassianNB-default,0.548151,0.549127,0.335694,0.394756,0.232479,0.779847,0.463560,0.399027,0.786834,0.736089,0.732580,0.608076,0.659341,0.311759,0.926214,0.321519
1,GuassianNB-best,0.548151,0.549050,0.335932,0.395044,0.232479,0.777656,0.463560,0.399027,0.786834,0.735504,0.733772,0.608076,0.658288,0.311759,0.927255,0.321519
2,LogisticRegression-default,0.556556,0.541332,0.427497,0.390310,0.268293,0.724696,0.523452,0.383731,0.791209,0.757601,0.580584,0.660277,0.624500,0.304348,0.757895,0.384252


ExtraTreesClassifier:
Similar to Random Forests, ExtraTrees is an ensemble ML approach that trains numerous decision trees and aggregates the results from the group of decision trees to output a prediction. However, there are few differences between Extra Trees and Random Forest.

Random Forest uses bagging to select different variations of the training data to ensure decision trees are sufficiently different. However, Extra Trees uses the entire dataset to train decision trees. As such, to ensure sufficient differences between individual decision trees, it RANDOMLY SELECTS the values at which to split a feature and create child nodes. In contrast, in a Random Forest, we use an algorithm to greedy search and select the value at which to split a feature. The came to the conclusion that ExtraTrees have higher bias and lower variance than Random Forest and that in terms of computational cost, Extra Trees is much faster than Random Forest (https://towardsdatascience.com/what-when-how-extratrees-classifier-c939f905851c)

In [ ]:
extress = ExtraTreesClassifier(n_jobs=-1, random_state=42)
extress.fit(X_train,y_train)

y_pred = extress.predict(X_val)
results_df = compute_metrics(y_val, y_pred, labels, "ExtraTrees-default", results_df)
print(confusion(y_val,y_pred))

In [ ]:
dump(extress, os.path.join(models_dir, "ExtraTrees_default.joblib"))

['/content/drive/MyDrive/Machine Learning Deliverable/models/ExtraTrees_default.joblib']

In [ ]:
scoring_cols = [
    'param_n_estimators', 'param_criterion', 'param_max_depth', 'param_min_samples_split', 'param_max_features', 'param_class_weight', 'mean_test_f1_mac', 'mean_test_acc'
]

Extra Trees hyperparameters:


1.   Criterion: *The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “log_loss” and “entropy” both for the Shannon information gain. (scikit-learn)*
Information gain uses the entropy measure as the impurity measure and splits a node such that it gives the most amount of information gain. Whereas Gini Impurity measures the divergences between the probability distributions of the target attribute’s values and splits a node such that it gives the least amount of impurity.
2.   n_estimators: *The number of trees in the forest.*
3.   max_depth: *The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.*
The deeper the tree, the more complex the model will become because there are more splits and it captures more information about the data and this is one of the root causes of overfitting. So, if the model is overfitting, reducing the number for max_depth is one way to combat overfitting. A very low depth will underfit sohow to find the best value.
4.   min_samples_split: *The minimum number of samples required to split an internal node*
5.   min_samples_leaf: *The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.*
Let's say we have a min_samples_split and the resulting split results in a leaf with 1 sample and you have specified min_samples_leaf as 2, then the min_samples_split will not be allowed. In other words, min_samples_leaf is always guaranteed no matter the min_samples_split value.
6.   max_features: *The number of features to consider when looking for the best split*.   We have 14 different features
7.   bootstrap: *Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.*
The default behaviour of ExtraTrees is to not boostrap and that is one of its main differences from RandomForest. In an ExtraTreesClassifier, we are drawing observations without replacement, so we will not have repetition of observations like in random forest. Setting it to True, makes its behavior more similar to RanfomForest
8.   class_weight: It is used to provide a weight or bias for each output class. This could be very usefull since we have an imbalanced dataset.
The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)). The “balanced_subsample” mode is the same as “balanced” except that weights are computed based on the bootstrap sample for every tree grown.




In [ ]:
extress = ExtraTreesClassifier(n_jobs=-1, random_state=42)

In [ ]:
init_time = time()
extress_cv = GridSearchCV(estimator=extress,
                   scoring=scoring_dict,
                   param_grid={
                       'n_estimators': [100, 500, 1000],
                       'criterion': ['gini'],
                       'max_depth': [2,4,6] + [None],
                       'min_samples_split': [2,4,6,8,10,12],
                       'max_features': ['sqrt', 'log2', None],
                       'class_weight': ['balanced', None],
                   },
                   cv=5,
                   return_train_score=False,
                   refit='f1_mac',
                   n_jobs=-1)
extress_5CV = extress_cv.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

3:20:15.931892


In [ ]:
pd.DataFrame(extress_5CV.cv_results_).sort_values(by='mean_test_f1_mac', ascending=False)[scoring_cols].head()

,param_n_estimators,param_criterion,param_max_depth,param_min_samples_split,param_max_features,param_class_weight,mean_test_f1_mac,mean_test_acc
205,500,gini,None,6,None,balanced,0.647977,0.633783
201,100,gini,None,4,None,balanced,0.647758,0.635498
208,500,gini,None,8,None,balanced,0.647669,0.631595
209,1000,gini,None,8,None,balanced,0.647668,0.632068
419,1000,gini,None,4,None,None,0.647633,0.638100


In [ ]:
best_params = extress_5CV.best_params_
best_params

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'min_samples_split': 6,
 'n_estimators': 500}

In [ ]:
with open(os.path.join(models_dir, "ExtraTrees_bestparams.txt"), "w") as f:
  f.write(json.dumps(best_params))
dump(extress_5CV, os.path.join(models_dir, "ExtraTrees_best.joblib"))

['/content/drive/MyDrive/Machine Learning Deliverable/models/ExtraTrees_best.joblib']

In [ ]:
y_pred = extress_5CV.predict(X_val)

results_df = compute_metrics(y_val, y_pred, labels, "ExtrTrees-best", results_df)
results_df.sort_values(by='F1_Avg', ascending=False,inplace=True)
results_df

,Algorithm,Accuracy,F1_Avg,F1_Dark Trap,F1_Emo,F1_Trap Metal,F1_techhouse,F1_Underground Rap,F1_Hiphop,F1_psytrance,F1_techno,F1_hardstyle,F1_trap,F1_trance,F1_RnB,F1_dnb,F1_Rap
5,ExtrTrees-best,0.637488,0.654126,0.469604,0.698061,0.326633,0.841683,0.450437,0.400000,0.911111,0.826923,0.851145,0.778455,0.831250,0.408493,0.960222,0.403743
4,ExtraTrees-default,0.619476,0.626704,0.454877,0.679831,0.246533,0.819939,0.478049,0.349398,0.883362,0.826873,0.809886,0.770031,0.781919,0.355342,0.938776,0.379045
0,LogisticRegression-best,0.565802,0.554464,0.433135,0.392573,0.341463,0.730382,0.520530,0.394366,0.829841,0.790102,0.587452,0.661856,0.657729,0.301691,0.759176,0.362205
1,GuassianNB-default,0.548151,0.549127,0.335694,0.394756,0.232479,0.779847,0.463560,0.399027,0.786834,0.736089,0.732580,0.608076,0.659341,0.311759,0.926214,0.321519
2,GuassianNB-best,0.548151,0.549050,0.335932,0.395044,0.232479,0.777656,0.463560,0.399027,0.786834,0.735504,0.733772,0.608076,0.658288,0.311759,0.927255,0.321519
3,LogisticRegression-default,0.556556,0.541332,0.427497,0.390310,0.268293,0.724696,0.523452,0.383731,0.791209,0.757601,0.580584,0.660277,0.624500,0.304348,0.757895,0.384252


In [ ]:
extress = ExtraTreesClassifier(n_jobs=-1, random_state=42, criterion='gini', max_depth=None, max_features=None, min_samples_split=6, class_weight="balanced", n_estimators=500)
scoring_cols = [
    'param_n_estimators', 'param_min_samples_leaf', 'mean_test_f1_mac', 'mean_test_acc'
]

In [ ]:
init_time = time()
extress_cv = GridSearchCV(estimator=extress,
                   scoring=scoring_dict,
                   param_grid={
                       'n_estimators': [500,600,700,800,900,1000],
                       'min_samples_leaf': [1,2,4,6,8,10,12],
                   },
                   cv=5,
                   return_train_score=False,
                   refit='f1_mac',
                   n_jobs=-1)
extress_5CV = extress_cv.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

1:08:54.051686


In [ ]:
pd.DataFrame(extress_5CV.cv_results_).sort_values(by='mean_test_f1_mac', ascending=False)[scoring_cols].head()

,param_n_estimators,param_min_samples_leaf,mean_test_f1_mac,mean_test_acc
1,600,1,0.647992,0.633605
0,500,1,0.647977,0.633783
6,500,2,0.647622,0.632659
5,1000,1,0.647287,0.632718
3,800,1,0.647252,0.632777


In [ ]:
best_params = extress_5CV.best_params_
best_params

{'min_samples_leaf': 1, 'n_estimators': 600}

In [ ]:
# results_df
# results_df = results_df[results_df['Algorithm'] != "ExtrTrees-best2"]
# results_df

In [ ]:
with open(os.path.join(models_dir, "ExtraTrees2_bestparams.txt"), "w") as f:
  f.write(json.dumps(best_params))
dump(extress_5CV, os.path.join(models_dir, "ExtraTrees2_best.joblib"))

['/content/drive/MyDrive/Machine Learning Deliverable/models/ExtraTrees2_best.joblib']

In [ ]:
y_pred = extress_5CV.predict(X_val)

# results_df = results_df.drop()
results_df = compute_metrics(y_val, y_pred, labels, "ExtrTrees-best2", results_df)
results_df.sort_values(by='F1_Avg', ascending=False,inplace=True)
results_df

,Algorithm,Accuracy,F1_Avg,F1_Dark Trap,F1_Emo,F1_Trap Metal,F1_techhouse,F1_Underground Rap,F1_Hiphop,F1_psytrance,F1_techno,F1_hardstyle,F1_trap,F1_trance,F1_RnB,F1_dnb,F1_Rap
6,ExtrTrees-best2,0.638329,0.655252,0.469866,0.699862,0.329177,0.842528,0.450266,0.400000,0.910180,0.830287,0.851958,0.778797,0.830986,0.418511,0.961111,0.400000
5,ExtrTrees-best,0.637488,0.654126,0.469604,0.698061,0.326633,0.841683,0.450437,0.400000,0.911111,0.826923,0.851145,0.778455,0.831250,0.408493,0.960222,0.403743
4,ExtraTrees-default,0.619476,0.626704,0.454877,0.679831,0.246533,0.819939,0.478049,0.349398,0.883362,0.826873,0.809886,0.770031,0.781919,0.355342,0.938776,0.379045
0,LogisticRegression-best,0.565802,0.554464,0.433135,0.392573,0.341463,0.730382,0.520530,0.394366,0.829841,0.790102,0.587452,0.661856,0.657729,0.301691,0.759176,0.362205
1,GuassianNB-default,0.548151,0.549127,0.335694,0.394756,0.232479,0.779847,0.463560,0.399027,0.786834,0.736089,0.732580,0.608076,0.659341,0.311759,0.926214,0.321519
2,GuassianNB-best,0.548151,0.549050,0.335932,0.395044,0.232479,0.777656,0.463560,0.399027,0.786834,0.735504,0.733772,0.608076,0.658288,0.311759,0.927255,0.321519
3,LogisticRegression-default,0.556556,0.541332,0.427497,0.390310,0.268293,0.724696,0.523452,0.383731,0.791209,0.757601,0.580584,0.660277,0.624500,0.304348,0.757895,0.384252


## Multi Layer Perceptron

A multilayer perceptron (MLP) is a feedforward neural network, with fully connected neurons with a nonlinear activation function, able to distinguish data not linearly separable. It is trained using the backpropagation method. The selection of hyperparameters is very important.

In [ ]:
mlp_default = MLPClassifier(random_state=42)
mlp_default.fit(X_train, y_train)

y_pred = mlp_default.predict(X_val)
results_df = compute_metrics(y_val, y_pred, labels, "MLP-default", results_df)
print(confusion(y_val,y_pred))

predicted        Dark Trap  Emo  Hiphop  Rap  RnB  Trap Metal  \
target                                                          
Dark Trap              461   10      18    2   27          39   
Emo                     11  232       7    3   31           8   
Hiphop                  47   21     235   29   93           4   
Rap                     17    3      23  124   25           2   
RnB                     33   34      77    3  187           1   
Trap Metal              78   11       6    2    3         110   
Underground Rap        108   12     134   32   71          51   
dnb                      4   11       7    0    3           1   
hardstyle               14    9       0    0    1           1   
psytrance                7    0       0    0    0           1   
techhouse                9    6       5    1    4           1   
techno                   1    0       0    0    0           0   
trance                  31   12       1    0    4           2   
trap                    1

In [ ]:
dump(mlp_default, os.path.join(models_dir, "MLP_default.joblib"))

['/content/drive/MyDrive/Machine Learning Deliverable/models/MLP_default.joblib']

In [ ]:
results_df.sort_values(by='F1_Avg', ascending=False,inplace=True)
results_df

,Algorithm,Accuracy,F1_Avg,F1_Dark Trap,F1_Emo,F1_Trap Metal,F1_techhouse,F1_Underground Rap,F1_Hiphop,F1_psytrance,F1_techno,F1_hardstyle,F1_trap,F1_trance,F1_RnB,F1_dnb,F1_Rap
6,ExtrTrees-best2,0.638329,0.655252,0.469866,0.699862,0.329177,0.842528,0.450266,0.400000,0.910180,0.830287,0.851958,0.778797,0.830986,0.418511,0.961111,0.400000
5,ExtrTrees-best,0.637488,0.654126,0.469604,0.698061,0.326633,0.841683,0.450437,0.400000,0.911111,0.826923,0.851145,0.778455,0.831250,0.408493,0.960222,0.403743
7,MLP-default,0.641210,0.644428,0.502179,0.638239,0.341085,0.800817,0.529730,0.402053,0.883682,0.825234,0.800000,0.758985,0.781457,0.413260,0.929163,0.416107
4,ExtraTrees-default,0.619476,0.626704,0.454877,0.679831,0.246533,0.819939,0.478049,0.349398,0.883362,0.826873,0.809886,0.770031,0.781919,0.355342,0.938776,0.379045
0,LogisticRegression-best,0.565802,0.554464,0.433135,0.392573,0.341463,0.730382,0.520530,0.394366,0.829841,0.790102,0.587452,0.661856,0.657729,0.301691,0.759176,0.362205
1,GuassianNB-default,0.548151,0.549127,0.335694,0.394756,0.232479,0.779847,0.463560,0.399027,0.786834,0.736089,0.732580,0.608076,0.659341,0.311759,0.926214,0.321519
2,GuassianNB-best,0.548151,0.549050,0.335932,0.395044,0.232479,0.777656,0.463560,0.399027,0.786834,0.735504,0.733772,0.608076,0.658288,0.311759,0.927255,0.321519
3,LogisticRegression-default,0.556556,0.541332,0.427497,0.390310,0.268293,0.724696,0.523452,0.383731,0.791209,0.757601,0.580584,0.660277,0.624500,0.304348,0.757895,0.384252


MLP Classifier hyperparameters:

*   hidden_layer_sizes: It specifies the number of layers and the neurons in each layer.
*   alpha: It is the L2 regularization term
*   activation: This is the activation function for the hidden layer. We test the ‘logistic’ sigmoid function, i.e.,  f(x) = 1 / (1 + exp(-x)), the hyperbolic tan function (‘tanh’), i.e., f(x) = tanh(x) and the rectified linear unit function (‘relu’), i.e. f(x) = max(0,x)
*   solver: The solver for weight optimization. We test stochastic gradient descent (‘sgd’) and ‘adam’ which is a sgd-based optimizer.
*   learning_rate: Learning rate for weight updates. We test ‘constant’ which is a constant learning rate with default value 0.001 and ‘invscaling’ which gradually decreases the learning rate at each time step using an inverse scaling exponent.








Important Note: The code below for fine tuning the hyperparameters of MLP were performed locally, because of Google Drive restrictions. Thus, it doesn't have similar outputs like in the previous tests. However, we still save the best model, its parameters and append it to the result table.

In [ ]:
sizes = [100*i for i in [1,3,5]]
sizes = sizes + [[100*i,100*i] for i in [1,3,5]]
sizes = sizes + [[100*i,100*i, 100*i] for i in [1,3,5]]
len(sizes), sizes

(9,
 [100,
  300,
  500,
  [100, 100],
  [300, 300],
  [500, 500],
  [100, 100, 100],
  [300, 300, 300],
  [500, 500, 500]])

In [ ]:
sizes=[100,200,[100,100],[200,200]]

In [ ]:
decays = [0.00001,0.0001,0.001,0.01,0.1,1]
len(decays), decays[:10]

(6, [1e-05, 0.0001, 0.001, 0.01, 0.1, 1])

In [ ]:
scoring_cols = [
    'param_hidden_layer_sizes', 'param_alpha', 'param_activation', 'param_solver', 'param_max_iter', 'param_learning_rate', 'mean_test_f1_mac', 'mean_test_acc'
]

In [ ]:
init_time = time()
mlp_cv = GridSearchCV(estimator=mlp_default,
                   scoring=scoring_dict,
                   param_grid={
                       'hidden_layer_sizes': sizes,
                       'alpha': decays,
                       'activation': ["logistic", "tanh", "relu"],
                       'solver': ["sgd", "adam"],
                       'max_iter': [200],
                       'learning_rate': ["constant", "invscaling"],
                   },
                   cv=5,
                   return_train_score=False,
                   refit='f1_mac',
                   n_jobs=-1)
mlp_5CV = mlp_cv.fit(X_train, y_train)
print(timedelta(seconds=(time() - init_time)))

In [ ]:
pd.DataFrame(mlp_5CV.cv_results_).sort_values(by='mean_test_f1_mac', ascending=False)[scoring_cols].head()

In [ ]:
best_params = mlp_5CV.best_params_
best_params

In [ ]:
y_pred = mlp_5CV.predict(X_val)

results_df = compute_metrics(y_val, y_pred, labels, "MLP-best", results_df)
results_df.sort_values(by='F1_Avg', ascending=False,inplace=True)
results_df

In [ ]:
results_df.to_csv('/content/drive/MyDrive/Machine Learning Deliverable/data/ML_results21_5.csv', index=False)

Test how ExtraTrees best model performs in the test unseen data.

In [ ]:
best_ex = ExtraTreesClassifier(n_jobs=-1, random_state=42, max_features=None, min_samples_split=6, n_estimators=700)
best = best_ex.fit(X_train, y_train)

In [ ]:
y_pred = best.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

   Dark Trap       0.55      0.61      0.58      1491
         Emo       0.74      0.68      0.71       528
      Hiphop       0.55      0.56      0.55       975
         Rap       0.56      0.50      0.53       597
         RnB       0.44      0.42      0.43       673
  Trap Metal       0.47      0.34      0.40       626
         dnb       0.98      0.97      0.98       794
   hardstyle       0.82      0.88      0.85       752
   psytrance       0.88      0.91      0.90       861
   techhouse       0.85      0.86      0.85       723
      techno       0.83      0.82      0.83       869
      trance       0.78      0.83      0.80       902
        trap       0.74      0.75      0.75       735

    accuracy                           0.71     10526
   macro avg       0.71      0.70      0.70     10526
weighted avg       0.70      0.71      0.70     10526



In [ ]:
confusion(y_test, y_pred)

predicted,Dark Trap,Emo,Hiphop,Rap,RnB,Trap Metal,dnb,hardstyle,psytrance,techhouse,techno,trance,trap
target,,,,,,,,,,,,,
Dark Trap,903,15,69,56,68,171,7,27,8,14,16,80,57
Emo,36,360,28,2,52,6,3,19,0,2,0,9,11
Hiphop,127,17,544,79,161,22,3,4,0,4,0,7,7
Rap,99,5,95,297,61,26,2,2,0,2,0,2,6
RnB,85,32,200,55,280,5,1,0,0,7,0,4,4
Trap Metal,268,25,32,32,5,215,1,8,0,5,0,7,28
dnb,2,7,9,0,2,1,773,0,0,0,0,0,0
hardstyle,13,5,1,0,0,0,0,660,11,0,0,0,62
psytrance,6,0,0,0,0,1,0,6,784,0,13,43,8
